In [1]:
import torch
import numpy as np
from yaspin import yaspin
import pickle
from tqdm.notebook import tqdm
import pandas as pd
import math
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, log_loss
from sentence_transformers import SentenceTransformer, models
from sklearn.model_selection import KFold
from transformers import RobertaModel, RobertaTokenizer

In [8]:
k_folds = 5

In [9]:
bert_model = SentenceTransformer('roberta-base')
bert_model.cuda()

Exception when trying to download https://sbert.net/models/roberta-base.zip. Response 404
SentenceTransformer-Model https://sbert.net/models/roberta-base.zip not found. Try to create it from scratch
Try to create Transformer Model roberta-base with mean pooling


SentenceTransformer(
  (0): Transformer(
    (auto_model): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(50265, 768, padding_idx=1)
        (position_embeddings): Embedding(514, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0): RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): RobertaSelfOutput(
                (dense): Linear(in_features=768, 

In [10]:
bert_model.max_seq_length = 1000

In [11]:
torch.manual_seed(10)

In [12]:
embed_size = 512
init_query_embed_size = 768
num_docs = 1
batch_size = 10
epochs = 20
max_len = 512

In [13]:
cos = torch.nn.CosineSimilarity()
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', truncation = True)

In [14]:
class SBERT(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.roberta = RobertaModel.from_pretrained('roberta-base')
        self.pooler = torch.nn.AvgPool1d(embed_size)
    def forward(self, ids, attention_mask, token_type_ids):
        output = self.roberta(input_ids = ids, attention_mask = attention_mask, token_type_ids = token_type_ids)
        output = output[0]
        output = self.pooler(output)
        return output

In [15]:
class GCN(torch.nn.Module):
    def __init__(self, n_h, input_features):
        super().__init__()
        W = torch.FloatTensor(input_features, n_h)
        self.weights = torch.nn.Parameter(W)
        self.relu = torch.nn.ReLU()
        stdv = 1. / math.sqrt(self.weights.size(1))
        self.weights.data.uniform_(-stdv, stdv)
        #torch.nn.init.xavier_uniform_(self.weights)
        bs = torch.FloatTensor(n_h)
        self.bias = torch.nn.Parameter(bs)
        self.bias.data.uniform_(-stdv, stdv)
        #torch.nn.init.xavier_uniform_(self.bias)
    
    def forward(self, nfs, A):
        out = torch.matmul(nfs, self.weights)
        out = torch.matmul(A, out)
        out = out + self.bias
        out = self.relu(out)
        return out

In [44]:
class AmbiguityNetwork(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.sbert = SBERT()
        self.gcn1 = GCN(20, embed_size)
        self.gcn2 = GCN(10, 20)
        self.gcn3 = GCN(10,10)
        self.dp = torch.nn.Dropout(p = 0.5)
        self.dense1 = torch.nn.Linear(in_features = num_docs*10 + init_query_embed_size, out_features = 64)
        self.dense2 = torch.nn.Linear(in_features = 64, out_features = 4)
    
    def forward(self, ids, attention_masks, token_type_ids, init_query):
        embeds = [self.sbert(id, attention_mask, token_type_id) for (id, attention_mask, token_type_id) in zip(ids, attention_masks, token_type_ids)] 
        embeds = [torch.squeeze(embed, dim = 2) for embed in embeds]
        embeds = torch.cat(embeds, dim = 0)
        emb_norm = embeds / embeds.norm(dim=1)[:, None]
        csml = torch.mm(emb_norm, emb_norm.transpose(0,1))
        A = torch.where(csml>0.1, csml, torch.zeros(csml.shape))
        S = torch.sum(A, axis = 1)
        D = torch.diag(torch.sum(A, axis = 1))
        D_ = torch.pow(D, -0.5)
        A_ = torch.chain_matmul(D_, A, D_)
        A = A_ + torch.eye(embeds.shape[0])
        out = self.gcn1(embeds, A)
        out = self.dp(out)
        out = self.gcn2(out, A)
        out = self.dp(out)
        out = self.gcn3(out, A)
        out = self.dp(out)
        print(out.shape)
        print(init_query.shape)
        out = torch.flatten(out, 1)
        out = torch.cat((out, init_query), dim = 1)
        out = self.dense1(out)
        out = self.dense2(out)
        return out

In [17]:
requests = pd.read_table('./data/train.tsv', sep = '\t', header = 0).drop_duplicates('topic_id')
requests

,topic_id,initial_request,topic_desc,clarification_need,facet_id,facet_desc,question_id,question,answer
0,1,Tell me about Obama family tree.,Find information on President Barack Obama\'s ...,2,F0001,"Find the TIME magazine photo essay ""Barack Oba...",Q00384,are you interested in seeing barack obamas family,yes am interested in obamas family
39,102,What is Fickle Creek Farm,Find general information about Fickle Creek Fa...,2,F0014,Find general information about Fickle Creek Fa...,Q00059,are you going to purchase anything there,i dont know yet i just want general info about...
84,105,Tell me about sonoma county medical services.,What medical services are available in Sonoma ...,2,F0025,What medical services are available in Sonoma ...,Q00457,are you interested in the human services depar...,no i am looking for doctors or hospitals in so...
120,108,Tell me about of Ralph Owen Brester.,Find biographical information about Ralph Owen...,1,F0037,Find biographical information about Ralph Owen...,Q00297,are you interested in learning more about ralp...,yes and his biography
159,109,I'm looking for information about mayo clinic ...,What medical services are available at the May...,2,F0040,What medical services are available at the May...,Q00256,are you interested in jobs at mayo clinic jack...,no im interested in services provided at mayo ...
...,...,...,...,...,...,...,...,...,...
9113,234,tell me about dark chocolate health benefits,What are the health benefits associated with e...,1,F0539,What are the health benefits associated with e...,Q00430,are you interested in the different candles da...,no im more interested in the health benefits o...
9127,238,Tell me bio of george bush sr.,Find biographies of US President George H.W. B...,2,F0552,Find biographies of US President George H.W. B...,Q00402,are you interested in the 1992 presidential el...,yes i would like to know about george bush srs...
9139,240,Tell me more about us presidents,Find a list of the full names of US presidents.,3,F0558,Find a list of the full names of US presidents.,Q00201,are you interested in finding out the middle n...,no i am interested in the names of all us pres...
9151,261,Tell me about folk remedies for a sore throat.,What folk remedies are there for soothing a so...,1,F0633,What folk remedies are there for soothing a so...,Q00208,are you interested in folk remedies from a spe...,no just any folk remedies for a sore throat


In [18]:
rdev = pd.read_table('./data/dev.tsv', sep = '\t', header = 0).drop_duplicates('topic_id')
rdev

,topic_id,initial_request,topic_desc,clarification_need,facet_id,facet_desc,question_id,question,answer
0,101,Find me information about the Ritz Carlton Lak...,Find information about the Ritz Carlton resort...,2,F0010,Find information about the Ritz Carlton resort...,Q00697,are you looking for a specific web site,yes for the ritz carlton resort at lake las vegas
60,106,I'm looking for universal animal cuts reviews,Find testimonials of Universal Animal Cuts nut...,3,F0028,Find testimonials of Universal Animal Cuts nut...,Q01481,did universal animal cuts work for you,i need testimonials on the universal animal cu...
102,107,tell me about cass county missouri,Find demographic information about Cass County...,3,F0031,Find demographic information about Cass County...,Q00086,are you interested in a list of homes for sale...,no i want demographic info for cass county mo
192,114,Tell about an adobe indian house?,How does one build an adobe house?,2,F0063,How does one build an adobe house?,Q00057,are you going to purchase any specific product...,maybe
231,123,What is von Willebrand Disease?,What is von Willebrand Disease?,3,F0100,What is von Willebrand Disease?,Q00284,are you interested in learning about treatment...,id like to know what it is first
267,128,Tell me about atypical squamous cells,What do atypical squamous cells mean on a pap ...,2,F0114,What do atypical squamous cells mean on a pap ...,Q00081,are you interested in a desciption of aytypica...,yes and it must be related to pap smear tests
303,133,all men are created equal,"Who said \""all men are created equal\""?",2,F0134,"Who said ""all men are created equal""?",Q00796,are you looking for declaration of independenc...,i am looking for who said the qoute provided
378,139,Tell me more about Rocky Mountain News,discussion of the impending sale of the Rocky ...,2,F0156,discussion of the impending sale of the Rocky ...,Q00334,are you interested in news archives,no i am intrested in the sale of arocky mounta...
426,142,Find me information about the sales tax in Ill...,information about the sales tax in Illinois: w...,2,F0170,information about the sales tax in Illinois: w...,Q00241,are you interested in how the illinois state t...,no i would like to know the rate and what it i...
471,164,I'm looking for information on hobby stores,What hobby stores carry trains?,4,F0259,What hobby stores carry trains?,Q00659,are you looking for a radiocontrolled plane,no im looking for trains


In [19]:
df = requests.append(rdev)

In [20]:
df

,topic_id,initial_request,topic_desc,clarification_need,facet_id,facet_desc,question_id,question,answer
0,1,Tell me about Obama family tree.,Find information on President Barack Obama\'s ...,2,F0001,"Find the TIME magazine photo essay ""Barack Oba...",Q00384,are you interested in seeing barack obamas family,yes am interested in obamas family
39,102,What is Fickle Creek Farm,Find general information about Fickle Creek Fa...,2,F0014,Find general information about Fickle Creek Fa...,Q00059,are you going to purchase anything there,i dont know yet i just want general info about...
84,105,Tell me about sonoma county medical services.,What medical services are available in Sonoma ...,2,F0025,What medical services are available in Sonoma ...,Q00457,are you interested in the human services depar...,no i am looking for doctors or hospitals in so...
120,108,Tell me about of Ralph Owen Brester.,Find biographical information about Ralph Owen...,1,F0037,Find biographical information about Ralph Owen...,Q00297,are you interested in learning more about ralp...,yes and his biography
159,109,I'm looking for information about mayo clinic ...,What medical services are available at the May...,2,F0040,What medical services are available at the May...,Q00256,are you interested in jobs at mayo clinic jack...,no im interested in services provided at mayo ...
...,...,...,...,...,...,...,...,...,...
2120,51,Tell me more about horse hooves,"Find information about horse hooves, their car...",2,F0858,Find information about horses' hooves and how ...,Q00025,are you asking about a horses feet,yes about horse hooves
2195,79,tell me about Voyager,Find information about the NASA Voyager spacec...,4,F0981,Find the homepage for the NASA Voyager mission.,Q00453,are you interested in the history of the voyager,no just take me to the homepage
2251,83,tell me about memory,Find information about human memory.,4,F0998,Find information about human memory.,Q00176,are you interested in computer memory,no
2293,256,Who is the patron saint of mental illness?,Who is the patron saint of mental illness?,1,F0609,Who is the patron saint of mental illness?,Q00304,are you interested in learning the back story ...,im interested in learning who it is


In [21]:
f = open('retrieved_docs_encoded_d2v_keywords.pkl', 'rb')

In [22]:
docs = {}
with yaspin().arc:
    while True:
        try:
            data = pickle.load(f)
            tid = list(data.keys())[0]
            docs[tid] = list(data.values())[0]
        except EOFError:
            break

0m 

In [23]:
f.close()

In [24]:
ks = list(docs.keys())

In [25]:
tids = requests['topic_id'].to_numpy(dtype = int)

In [26]:
tids = np.intersect1d(tids, np.array(ks))

In [27]:
len(tids)

187

In [28]:
inputs = []
for tid in tids:
    inputs.append(docs[tid]['docs'][:num_docs])

In [29]:
inputs[0]

['columbia floridageorgia guam hawaii idaho illinois indiana iowa kansas kentucky louisiana mainemaryland massachusetts michigan minnesota mississippi missouri montana nebraskanevada new hampshire new jersey new mexico new york north carolina north dakotanorthern mariana islands ohio oklahoma oregon pennsylvania puerto ricorhode island south carolina south dakota tennessee texas utah vermont virginiavirgin islands washington. abroad alabama alaska american samoa arizona arkansascalifornia canal zone colorado connecticut delaware district. vital records international vital records adoption resources genealogy help investigators search government records first name. washington west virginia wisconsin wyoming order vital records online search birth. global tree searcher search submitted gedcom files. locating vital records surname searches u. social security death index surname searches search. rootsweb surname list one great genealogy site. social security death index kindred konnections

In [30]:
labels = torch.LongTensor(torch.zeros(len(tids)).long())
init_qrs = torch.zeros((len(tids), init_query_embed_size))
ids = torch.LongTensor(len(tids), num_docs, max_len)
masks = torch.LongTensor(len(tids), num_docs, max_len)
token_type_ids = torch.LongTensor(len(tids), num_docs, max_len)

In [31]:
ids.shape

torch.Size([187, 1, 512])

In [32]:
for i in tqdm(range(len(tids))):
    tid = tids[i]
    documents = inputs[i]
    tokens = [tokenizer.encode_plus(doc, None, add_special_tokens = True, max_length = max_len, padding = 'max_length', return_token_type_ids = True, truncation = True) for doc in documents]
    idlist = [torch.tensor(token['input_ids'], dtype = torch.long) for token in tokens]
    torch.cat(idlist, out = ids[i])
    masklist = [torch.tensor(token['attention_mask'], dtype = torch.long) for token in tokens]
    torch.cat(masklist, out = masks[i])
    tokentypelist = [torch.tensor(token['token_type_ids'], dtype = torch.long) for token in tokens]
    torch.cat(tokentypelist, out = token_type_ids[i])
    labels[i] = df.loc[df['topic_id'] == tid]['clarification_need'].to_numpy(dtype = np.int_)[0]
    init_qr = df.loc[df['topic_id'] == tid]['initial_request'].to_numpy(dtype = str)[0]
    init_qrs[i] = torch.Tensor(bert_model.encode(init_qr))

In [33]:
class InputDataset(torch.utils.data.Dataset):
    def __init__(self, ids, masks, token_type_ids, labels, init_qrs):
        self.ids = ids
        self.masks = masks
        self.token_type_ids = token_type_ids
        self.labels = torch.LongTensor(labels.long())
        self.init_qrs = init_qrs
        
    def __len__(self):
        return len(self.ids)
    
    def __getitem__(self, idx):
        return self.ids[idx], self.masks[idx], self.token_type_ids[idx], self.labels[idx], self.init_qrs[idx]

In [34]:
dev_tids = rdev['topic_id'].to_numpy(dtype = int)

In [35]:
dev_labels = rdev['clarification_need'].to_numpy(dtype = int)

In [36]:
dev_inputs = []
for tid in dev_tids:
    dev_inputs.append(docs[tid]['docs'][:num_docs])
dev_inputs = np.array(dev_inputs)

In [37]:
dev_init_qrs = torch.zeros((len(dev_tids), init_query_embed_size))
dev_labels = torch.LongTensor(torch.zeros(len(dev_tids)).long())
dev_ids = torch.LongTensor(len(dev_tids), num_docs, max_len)
dev_masks = torch.LongTensor(len(dev_tids), num_docs, max_len)
dev_token_type_ids = torch.LongTensor(len(dev_tids), num_docs, max_len)

In [38]:
for i in tqdm(range(len(dev_tids))):
    tid = dev_tids[i]
    docs = dev_inputs[i]
    tokens = [tokenizer.encode_plus(doc, None, add_special_tokens = True, max_length = max_len, padding = 'max_length', return_token_type_ids = True, truncation = True) for doc in docs]
    dev_id_list = [torch.tensor(token['input_ids'], dtype = torch.long) for token in tokens]
    torch.cat(dev_id_list, out = dev_ids[i])
    dev_mask_list = [torch.tensor(token['attention_mask'], dtype = torch.long) for token in tokens]
    torch.cat(dev_mask_list, out = dev_masks[i])
    dev_token_type_id_list = [torch.tensor(token['token_type_ids'], dtype = torch.long) for token in tokens]
    torch.cat(dev_token_type_id_list, out = dev_token_type_ids[i])
    dev_labels[i] = df.loc[df['topic_id'] == tid]['clarification_need'].to_numpy(dtype = np.int_)[0]
    init_qr = rdev.loc[rdev['topic_id'] == tid]['initial_request'].to_numpy(dtype = str)[0]
    dev_init_qrs[i] = torch.Tensor(bert_model.encode(init_qr))

In [39]:
train_dataset = InputDataset(ids, masks, token_type_ids, labels, init_qrs)
dev_dataset = InputDataset(dev_ids, dev_masks, dev_token_type_ids, dev_labels, dev_init_qrs)
dataset = torch.utils.data.ConcatDataset([train_dataset, dev_dataset])
kfold = KFold(n_splits = k_folds, shuffle = True)

In [42]:
def train(epochs, batch_size):
    loss_fn = torch.nn.CrossEntropyLoss()
    results = {}
    for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):
        print(f"Fold: {fold}")
        train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
        test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
        train_loader = torch.utils.data.DataLoader(dataset, batch_size = batch_size, sampler = train_subsampler)
        test_loader = torch.utils.data.DataLoader(dataset, batch_size = batch_size, sampler = test_subsampler)
        model = AmbiguityNetwork()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay = 0.1)
        curr_count = 0
        acc = 0
        curr_loss = 0
        for epoch in tqdm(range(epochs)):
            model.train()
            for idx, data in enumerate(train_loader):
                ids, masks, token_type_ids, tgts, init_query = data
                tgts = tgts - 1
                tgts = torch.LongTensor(tgts)
                preds = model(ids, masks, token_type_ids, init_query)
                loss = loss_fn(preds, tgts)
                m = torch.nn.Softmax()
                npreds = m(preds).detach().numpy()
                class_preds = np.argmax(npreds, axis = 1)
                acc += accuracy_score(tgts.detach().numpy(), class_preds)
                curr_loss += loss.item()
                curr_count += 1
                model.zero_grad()
                loss.backward()
                optimizer.step()
        curr_loss = curr_loss/curr_count
        acc = acc/curr_count

        with torch.set_grad_enabled(False):
            model.eval()
            val_acc = 0
            val_loss = 0
            val_prec = 0
            val_rec = 0
            val_f1 = 0
            total = 0
            count = 0
            for dev_idx, dev_data in enumerate(test_loader):
                ids, masks, token_type_ids, dev_labels, init_query = dev_data
                val_preds = model(ids, masks, token_type_ids, init_query)
                m = torch.nn.Softmax()
                val_npreds = m(val_preds).detach().numpy()
                class_preds = np.argmax(val_npreds, axis = 1)
                class_preds = class_preds + 1
                val_acc += accuracy_score(dev_labels, class_preds)
                val_loss += log_loss(dev_labels, val_npreds, labels = [1,2,3,4])
                val_prec += precision_score(dev_labels, class_preds, average = 'weighted')
                val_rec += recall_score(dev_labels, class_preds, average = 'weighted')
                val_f1 += f1_score(dev_labels, class_preds, average = 'weighted')
                count += 1
        results[fold] = {
            'training_loss': curr_loss,
            'training_acc': acc,
            'validation_loss': val_loss/count,
            'validation_acc': val_acc/count,
            'precision': val_prec/count,
            'recall': val_rec/count,
            'f1': val_f1/count
        }
        print(f"Loss: {curr_loss}, training acc: {acc}, validation_loss: {val_loss/count}, validation acc: {val_acc/count}")

    train_acc = 0
    train_loss = 0
    val_acc = 0
    val_loss = 0
    prec = 0
    f1 = 0
    rec = 0
    for key, value in results.items():
        train_acc += value['training_acc']
        train_loss += value['training_loss']
        val_acc += value['validation_acc']
        val_loss += value['validation_loss']
        prec += value['precision']
        rec += value['recall']
        f1 += value['f1']
    print('Final Metrics:')
    print(f"Training loss: {train_loss/k_folds}, Training acc: {train_acc/k_folds}, Validation loss: {val_loss/k_folds}, Validation acc: {val_acc/k_folds}, Precision: {prec/k_folds}, Recall: {rec/k_folds}, F1: {f1/k_folds}")

In [45]:
train(epochs, batch_size)

Fold: 0


torch.Size([10, 10])
torch.Size([10, 768])


<ipython-input-42-0b0d299e6c7a>:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  npreds = m(preds).detach().numpy()


torch.Size([10, 10])
torch.Size([10, 768])



KeyboardInterrupt: 

In [2]:
a = torch.randn(20, 30)
b = torch.randn(20, 30)

In [12]:
c = [a,b]
c

[tensor([[ 4.9743e-02,  1.2899e+00,  1.9881e-01,  1.9279e+00,  1.1709e+00,
           2.6120e+00, -1.0697e-01, -8.9150e-01,  1.4650e+00, -1.3107e+00,
           9.5090e-02, -5.8401e-01,  1.4513e-01,  6.2530e-01,  2.5424e-01,
          -1.3467e-02,  1.1158e+00,  2.4294e+00, -1.1893e-01, -2.0777e-01,
          -2.0487e+00, -1.8414e+00,  7.6088e-01, -4.4129e-01, -7.7623e-01,
          -1.3715e+00,  1.1704e-01, -9.9925e-02,  9.4874e-01,  4.1683e-02],
         [ 1.9142e-01,  8.1798e-01,  9.7078e-01, -6.2378e-01,  1.5227e+00,
           2.3703e+00, -8.6472e-01, -5.4576e-01,  1.7926e+00,  4.0773e-01,
          -6.6583e-01,  1.5273e+00, -1.5159e+00, -1.0065e-01, -2.2580e+00,
           1.0232e+00,  4.3127e-01,  7.2379e-01,  6.3327e-01, -1.4140e+00,
           1.3679e+00, -1.0119e+00,  1.7948e+00, -1.8274e+00, -1.5646e+00,
           7.2162e-01,  5.6718e-01,  4.4328e-01, -4.5884e-01,  7.6955e-01],
         [ 4.9090e-01, -1.7432e+00,  8.5835e-01, -1.7602e+00, -2.7837e-01,
          -1.3569e+00, 

In [20]:
d = torch.cat(c, dim = 0).reshape((2,20,30))
d.shape

torch.Size([2, 20, 30])

In [19]:
d.norm(dim = 2)[:, None].shape

torch.Size([2, 1, 20])

In [34]:
sm = torch.sum(d,axis = 2)

In [37]:
sm

tensor([[ 5.4354,  5.2253, -6.4804, -0.4859, -4.2469,  0.2175,  1.5483, -3.3177,
         -0.8195, -2.7294,  3.8151,  6.1827,  4.4154, -1.2265,  7.1864,  1.5278,
         -3.7656, -7.4804, -3.1971, -5.5668],
        [-7.3694, -1.7769,  1.1039,  1.4763,  5.7717, -8.9476, -8.2502, -1.9240,
         -1.8876,  1.3514,  3.9892,  1.0080,  0.0784, -3.8130, -5.6585, -5.3679,
          1.5412, -1.3541, -3.7464, -4.2420]])

In [40]:
torch.diag_embed(sm)

tensor([[[ 5.4354,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  5.2253,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000, -6.4804,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000, -0.4859,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000, -4.2469,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          